##### Copyright 2018 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 오토그래프(AutoGraph): Easy control flow for graphs

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/autograph"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />TensorFlow.org 에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/autograph.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Google Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/autograph.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />깃헙(GitHub) 소스 보기</a>
  </td>
</table>

Note: 이 문서는 텐서플로 커뮤니티에서 번역했습니다. 커뮤니티 번역 활동의 특성상 정확한 번역과 최신 내용을 반영하기 위해 노력함에도
불구하고 [공식 영문 문서](https://www.tensorflow.org/?hl=en)의 내용과 일치하지 않을 수 있습니다.
이 번역에 개선할 부분이 있다면
[tensorflow/docs](https://github.com/tensorflow/docs) 깃헙 저장소로 풀 리퀘스트를 보내주시기 바랍니다.
문서 번역이나 리뷰에 참여하려면
[docs-ko@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ko)로
메일을 보내주시기 바랍니다.

[오토그래프(AutoGraph)](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/) 모듈은 일반적인 파이썬을 사용해서 복잡한 그래프 코드를 작성하는데 도움을 줄 수 있습니다. 더 나아가, 이 모듈은 자동으로 여러분의 코드를  [텐서플로 그래프 코드](https://www.tensorflow.org/guide/graphs)로 변환해 줄 수 있습니다.  이 모듈은 이미 파이썬 언어의 많은 부분을 지원하고 있고, 앞으로 더 많은 부분이 지원될 것입니다. 파이썬 언어의 성질 중 지원되는 부분을 확인하려면, [오토그래프 지원 범위](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/LIMITATIONS.md)를 참고하세요. 



## 설정

TensorFlow, AutoGraph 와 필요한 모듈들을 import합니다.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
layers = tf.keras.layers
from tensorflow import contrib
autograph = contrib.autograph

import numpy as np
import matplotlib.pyplot as plt

결과 시연을 위해서 [즉시 실행(eager execution)](https://www.tensorflow.org/guide/eager)이 가능하게 해줍니다. 하지만 오토그래프 모듈은 즉시 실행과 [그래프 실행(graph execution)](https://www.tensorflow.org/guide/graphs)환경에서 모두 동작합니다:

In [0]:
tf.enable_eager_execution()

Note: 오토그래프 코드는 본래 그래프 실행을 하는 동안 동작하도록 만들어졌습니다.  즉시 실행을 설정했을 경우, 명시적으로 그래프를 생성하거나 (본 예시에서 볼 수 있듯이) 혹은 `tf.contrib.eager.defun`을 사용해야합니다.


## 파이썬 제어 흐름(control flow)을 자동으로 변환하기 Automatically convert Python control flow

오토그래프 모듈은 파이썬 언어의 대부분을 텐서플로 그래프 빌딩(building) 코드로 변환합니다. 

Note: 실제로 활용할 때 배치(batching)는 수행을 위해 필수적입니다. 오토그래프 모듈로 변환하기 좋은 코드는 _배치(batch)_ 수준에서 제어 흐름이 결정되는 코드입니다. 만일 개별적인 _예제(example)_ 수준에서 제어 흐름이 결정된다면, 제어 흐름 논리가 적용되는 동안 수행을 유지하기 위해서 예제들에 인덱스를 붙이고 배치화해야합니다. 

오토그래프 모듈은 다음과 같은 함수를:

In [0]:
def square_if_positive(x):
  if x > 0:
    x = x * x
  else:
    x = 0.0
  return x

그래프 빌딩을 포함하는 함수로 변환합니다:

In [0]:
print(autograph.to_code(square_if_positive))

즉시 실행 코드는 `tf.Graph` 내에서 그래프 실행 코드와 같은 결과값을 보이며 동작할 수 있습니다. 또한 그래프 실행도 가능하다는 장점을 갖추고 있습니다.

In [0]:
print('Eager results: %2.2f, %2.2f' % (square_if_positive(tf.constant(9.0)),
                                       square_if_positive(tf.constant(-9.0))))

그래프 버전을 생성한 후에 호출합니다:

In [0]:
tf_square_if_positive = autograph.to_graph(square_if_positive)

with tf.Graph().as_default():
  # The result works like a regular op: takes tensors in, returns tensors.
  # You can inspect the graph using tf.get_default_graph().as_graph_def()
  g_out1 = tf_square_if_positive(tf.constant( 9.0))
  g_out2 = tf_square_if_positive(tf.constant(-9.0))
  with tf.Session() as sess:
    print('Graph results: %2.2f, %2.2f\n' % (sess.run(g_out1), sess.run(g_out2)))

오토그래프 모듈은 `while`, `for`, `if`, `break`와 `return` 같은 일반적인 파이썬 구문과 중첩(nesting)을 지원합니다. 다음의 파이썬 함수와 그를 변환한 복잡한 그래프 버전을 비교해볼 수 있습니다:

In [0]:
# Continue in a loop
def sum_even(items):
  s = 0
  for c in items:
    if c % 2 > 0:
      continue
    s += c
  return s

print('Eager result: %d' % sum_even(tf.constant([10,12,15,20])))

tf_sum_even = autograph.to_graph(sum_even)

with tf.Graph().as_default(), tf.Session() as sess:
    print('Graph result: %d\n\n' % sess.run(tf_sum_even(tf.constant([10,12,15,20]))))

In [0]:
print(autograph.to_code(sum_even))

## 데코레이터

파이썬 코드로 작성된 원래의 함수에 접근할 필요가 없다면, `convert` 데코레이터를 사용합니다.

In [0]:
@autograph.convert(optional_features=autograph.Feature.ALL)
def fizzbuzz(i, n):
  while i < n:
    msg = ''
    if i % 3 == 0:
      msg += 'Fizz'
    if i % 5 == 0:
      msg += 'Buzz'
    if msg == '':
      msg = tf.as_string(i)
    print(msg)
    i += 1
  return i

with tf.Graph().as_default():
  final_i = fizzbuzz(tf.constant(10), tf.constant(16))
  # The result works like a regular op: takes tensors in, returns tensors.
  # You can inspect the graph using tf.get_default_graph().as_graph_def()
  with tf.Session() as sess:
    sess.run(final_i)



## 예제 Examples

파이썬 언어의 몇 가지 유용한 구문들을 실습해보겠습니다.


### Assert

오토그래프 모듈은 파이썬의  `assert`문을  `tf.Assert` 로 변환합니다:

In [0]:
@autograph.convert(optional_features=autograph.Feature.ALL)
def inverse(x):
  assert x != 0.0, 'Do not pass zero!'
  return 1.0 / x

with tf.Graph().as_default(), tf.Session() as sess:
  try:
    print(sess.run(inverse(tf.constant(0.0))))
  except tf.errors.InvalidArgumentError as e:
    print('Got error message:\n    %s' % e.message)

### Print

원한다면, `tf.function`의 의존성 관리 (dependency management) 기능을 사용하여 그래프 내에서 파이썬의 `print` 함수를 사용할 수 있습니다:

In [0]:
@tf.function(experimental_autograph_options=autograph.Feature.BUILTIN_FUNCTIONS)
def count(n):
  i = 0
  while i < n:
    print(i)
    i += 1
  return n

with tf.Graph().as_default(), tf.Session() as sess:
    sess.run(count(tf.constant(5)))

### Lists

루프 내에서 리스트를 추가합니다 (텐서 리스트인 ops가 자동으로 생성됩니다):

In [0]:
@autograph.convert(optional_features=autograph.Feature.LISTS)
def arange(n):
  z = []
  # We ask you to tell us the element dtype of the list
  autograph.set_element_type(z, tf.int32)

  for i in tf.range(n):
    z.append(i)
  # when you're done with the list, stack it
  # (this is just like np.stack)
  return autograph.stack(z)


with tf.Graph().as_default(), tf.Session() as sess:
    print(sess.run(arange(tf.constant(10))))

### 중첩된 제어 흐름(Nested control flow)

In [0]:
@autograph.convert(optional_features=autograph.Feature.EQUALITY_OPERATORS)
def nearest_odd_square(x):
  if x > 0:
    x = x * x
    if x % 2 == 0:
      x = x + 1
  return x

with tf.Graph().as_default():
  with tf.Session() as sess:
    print(sess.run(nearest_odd_square(tf.constant(4))))
    print(sess.run(nearest_odd_square(tf.constant(5))))
    print(sess.run(nearest_odd_square(tf.constant(6))))

### While 문

In [0]:
@autograph.convert()
def square_until_stop(x, y):
  while x < y:
    x = x * x
  return x

with tf.Graph().as_default():
  with tf.Session() as sess:
    print(sess.run(square_until_stop(tf.constant(4), tf.constant(100))))

### For 문

In [0]:
@autograph.convert(optional_features=autograph.Feature.LISTS)
def squares(nums):

  result = []
  autograph.set_element_type(result, tf.int64)

  for num in nums:
    result.append(num * num)

  return autograph.stack(result)

with tf.Graph().as_default():
  with tf.Session() as sess:
    print(sess.run(squares(tf.constant(np.arange(10)))))

### Break

In [0]:
@autograph.convert()
def argwhere_cumsum(x, threshold):
  current_sum = 0.0
  idx = 0
  for i in tf.range(len(x)):
    idx = i
    if current_sum >= threshold:
      break
    current_sum += x[i]
  return idx

N = 10
with tf.Graph().as_default():
  with tf.Session() as sess:
    idx = argwhere_cumsum(tf.ones(N), tf.constant(float(N/2)))
    print(sess.run(idx))

## `tf.Keras`와의 상호 운용성 Interoperation with `tf.Keras`

지금까지 기초를 보았고, 이번에는 오토그래프를 이용해 모델 요소들을 만들어보겠습니다. 

`autograph`를 `tf.keras`에 통합하는 것은 상대적으로 간단합니다.


### 무상태 함수 (Stateless functions)

아래에 나와있는 `collatz`와 같은 무상태 함수들을 케라스 모델에 포함시키는 가장 쉬운 방법은 `tf.keras.layers.Lambda` 를 이용해서 한 레이어에 감싸는 것(wrap)입니다.

In [0]:
import numpy as np

@autograph.convert(optional_features=autograph.Feature.ALL)
def collatz(x):
  x = tf.reshape(x,())
  assert x > 0
  n = tf.convert_to_tensor((0,))
  while x != 1:
    n += 1
    if x % 2 == 0:
      x = x // 2
    else:
      x = 3 * x + 1

  return n

with tf.Graph().as_default():
  model = tf.keras.Sequential([
    tf.keras.layers.Lambda(collatz, input_shape=(1,), output_shape=())
  ])

  result = model.predict(np.array([6171]))
  print(result)

### 사용자 정의 레이어와 모델들

<!--TODO(markdaoust) link to full examples  or these referenced models.-->

오토그래프를 케라스 레이어와 모델에서 사용하는 가장 쉬운 방법은 `call` 방식에 `@autograph.convert()`를 적용하는 것입니다. 이처럼 클래스를 기반으로 코드를 작성하는 방법을 자세히 보려면 [텐서플로 케라스 가이드](https://tensorflow.org/guide/keras#build_advanced_models)를 참고하세요

다음은 [확률적 네트워크 깊이(stochastic network depth)](https://arxiv.org/abs/1603.09382) 기법의 간단한 예시입니다:

In [0]:
# `K` is used to check if we're in train or test mode.
K = tf.keras.backend

class StochasticNetworkDepth(tf.keras.Sequential):
  def __init__(self, layers, pfirst=1.0, plast=0.5,**kwargs):
    self.pfirst = pfirst
    self.plast = plast
    super(StochasticNetworkDepth, self).__init__(layers,**kwargs)

  def build(self, input_shape):
    self.depth = len(self.layers)
    self.plims = np.linspace(self.pfirst, self.plast, self.depth + 1)[:-1]
    super(StochasticNetworkDepth, self).build(input_shape.as_list())

  @autograph.convert(optional_features=autograph.Feature.ALL)
  def call(self, inputs):
    training = tf.cast(K.learning_phase(), dtype=bool)
    if not training:
      count = self.depth
      return super(StochasticNetworkDepth, self).call(inputs), count

    p = tf.random_uniform((self.depth,))

    keeps = (p <= self.plims)
    x = inputs

    count = tf.reduce_sum(tf.cast(keeps, tf.int32))
    for i in range(self.depth):
      if keeps[i]:
        x = self.layers[i](x)

    # return both the final-layer output and the number of layers executed.
    return x, count

이것을 MNIST 모양의 데이터에 적용해보겠습니다:

In [0]:
train_batch = np.random.randn(64, 28, 28, 1).astype(np.float32)

확률적 깊이 모델 (stochastic depth model) 내에서 여러 장의 `conv` 레이어를 쌓아올립니다:

In [0]:
with tf.Graph().as_default() as g:
  model = StochasticNetworkDepth(
      [
        layers.Conv2D(filters=16, activation=tf.nn.relu,
                  kernel_size=(3, 3), padding='same')
        for n in range(20)
      ],
      pfirst=1.0, plast=0.5
  )

  model.build(tf.TensorShape((None, None, None, 1)))

  init = tf.global_variables_initializer()

학습 모드와 테스트 모드에서 예상한 대로 동작하는지 확인하기 위해 다음을 실행해봅니다:

In [0]:
# Use an explicit session here so we can set the train/test switch, and
# inspect the layer count returned by `call`
with tf.Session(graph=g) as sess:
  init.run()

  for phase, name in enumerate(['test','train']):
    K.set_learning_phase(phase)
    result, count = model(tf.convert_to_tensor(train_batch, dtype=tf.float32))

    result1, count1 = sess.run((result, count))
    result2, count2 = sess.run((result, count))

    delta = (result1 - result2)
    print(name, "sum abs delta: ", abs(delta).mean())
    print("    layers 1st call: ", count1)
    print("    layers 2nd call: ", count2)
    print()

## 응용 예제: 그래프 내의 학습 루프

이전 섹션에서는 오토그래프 모듈이 케라스 레이어와 모델들에서 사용될 수 있다는 것을 보여주었습니다. 반대로 케라스 모델 또한 오토 그래프 코드 내에서 사용될 수 있습니다.

오토그래프 내에서 제어 흐름을 작성하는 것이 쉽기 때문에, 텐서플로 그래프로 학습 루프를 실행하는 것 또한 쉽습니다. Since writing control flow in AutoGraph is easy, running a training loop in a TensorFlow graph should also be easy.

이 예시는 어떻게 간단한 케라스 모델을 MNIST 데이터에 학습시킬 수 있는지 그래프 내에서 진행되는 전체 학습과정 (배치를 로드하고, 그래디언트를 계산하고, 파라미터를 업데이트하고, 검증 정확도를 계산하고, 극점에 수렴할 때까지 반복하는 과정) 과 함께 보여줍니다.

### 데이터셋 다운로드

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

### 모델 정의하기

In [0]:
def mlp_model(input_shape):
  model = tf.keras.Sequential((
      tf.keras.layers.Dense(100, activation='relu', input_shape=input_shape),
      tf.keras.layers.Dense(100, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')))
  model.build()
  return model


def predict(m, x, y):
  y_p = m(tf.reshape(x, (-1, 28 * 28)))
  losses = tf.keras.losses.categorical_crossentropy(y, y_p)
  l = tf.reduce_mean(losses)
  accuracies = tf.keras.metrics.categorical_accuracy(y, y_p)
  accuracy = tf.reduce_mean(accuracies)
  return l, accuracy


def fit(m, x, y, opt):
  l, accuracy = predict(m, x, y)
  # Autograph automatically adds the necessary `tf.control_dependencies` here.
  # (Without them nothing depends on `opt.minimize`, so it doesn't run.)
  # This makes it much more like eager-code.
  opt.minimize(l)
  return l, accuracy


def setup_mnist_data(is_training, batch_size):
  if is_training:
    ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
    ds = ds.shuffle(batch_size * 10)
  else:
    ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

  ds = ds.repeat()
  ds = ds.batch(batch_size)
  return ds


@autograph.convert()
def get_next_batch(ds):
  itr = tf.compat.v1.data.make_one_shot_iterator(ds)
  image, label = itr.get_next()
  x = tf.to_float(image) / 255.0
  y = tf.one_hot(tf.squeeze(label), 10)
  return x, y

### 학습 루프 정의하기

In [0]:
# Use `recursive = True` to recursively convert functions called by this one.
@autograph.convert(recursive=True, optional_features=autograph.Feature.ALL)
def train(train_ds, test_ds, hp):
  m = mlp_model((28 * 28,))
  opt = tf.train.AdamOptimizer(hp.learning_rate)

  # We'd like to save our losses to a list. In order for AutoGraph
  # to convert these lists into their graph equivalent,
  # we need to specify the element type of the lists.
  train_losses = []
  autograph.set_element_type(train_losses, tf.float32)
  test_losses = []
  autograph.set_element_type(test_losses, tf.float32)
  train_accuracies = []
  autograph.set_element_type(train_accuracies, tf.float32)
  test_accuracies = []
  autograph.set_element_type(test_accuracies, tf.float32)

  # This entire training loop will be run in-graph.
  i = tf.constant(0)
  while i < hp.max_steps:
    train_x, train_y = get_next_batch(train_ds)
    test_x, test_y = get_next_batch(test_ds)

    step_train_loss, step_train_accuracy = fit(m, train_x, train_y, opt)
    step_test_loss, step_test_accuracy = predict(m, test_x, test_y)
    if i % 50 == 0:
      print('Step', i, 'train loss:', step_train_loss, 'test loss:',
            step_test_loss, 'train accuracy:', step_train_accuracy,
            'test accuracy:', step_test_accuracy)
    train_losses.append(step_train_loss)
    test_losses.append(step_test_loss)
    train_accuracies.append(step_train_accuracy)
    test_accuracies.append(step_test_accuracy)
    i += 1

  # We've recorded our loss values and accuracies
  # to a list in a graph with AutoGraph's help.
  # In order to return the values as a Tensor,
  # we need to stack them before returning them.
  return (autograph.stack(train_losses), autograph.stack(test_losses),
          autograph.stack(train_accuracies), autograph.stack(test_accuracies))

그래프를 빌딩하고 학습 루프를 실행시킵니다:

In [0]:
with tf.Graph().as_default() as g:
  hp = tf.contrib.training.HParams(
      learning_rate=0.005,
      max_steps=500,
  )
  train_ds = setup_mnist_data(True, 50)
  test_ds = setup_mnist_data(False, 1000)
  (train_losses, test_losses, train_accuracies,
   test_accuracies) = train(train_ds, test_ds, hp)

  init = tf.global_variables_initializer()

with tf.Session(graph=g) as sess:
  sess.run(init)
  (train_losses, test_losses, train_accuracies,
   test_accuracies) = sess.run([train_losses, test_losses, train_accuracies,
                                test_accuracies])


In [0]:
plt.title('MNIST train/test losses')
plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.legend()
plt.xlabel('Training step')
plt.ylabel('Loss')
plt.show()
plt.title('MNIST train/test accuracies')
plt.plot(train_accuracies, label='train accuracy')
plt.plot(test_accuracies, label='test accuracy')
plt.legend(loc='lower right')
plt.xlabel('Training step')
plt.ylabel('Accuracy')
plt.show()